In [1]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session

## Set Spark Session Configuration

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

In [3]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

In [4]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))
       )

In [5]:
# https://spark.apache.org/docs/latest/configuration.html#memory-management
spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.local.dir', '/home/ec2-user/SageMaker/tmp')
    .config("spark.executor.memory", "70g")
    .config("spark.driver.memory", "50g")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size","50g")
    .appName("sapient")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 00:43:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/04 00:43:27 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/04 00:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/04 00:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Functions to Load and Read Data

In [6]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

In [7]:
# schemas to reduce read times for spark 
bro_schema = StructType([
        StructField('ts', StringType(), True),
        StructField('uid', StringType(), True),
        StructField('id.orig_h', StringType(), True),
        StructField('id.orig_p', IntegerType(), True),
        StructField('id.resp_', StringType(), True),
        StructField('id.resp_p', IntegerType(), True),
        StructField('proto', StringType(), True),
        StructField('service', StringType(), True),
        StructField('duration', StringType(), True),
        StructField('orig_bytes', IntegerType(), True),
        StructField('resp_bytes', IntegerType(), True),
        StructField('conn_state', StringType(), True),
        StructField('local_orig', StringType(), True),
        StructField('local_resp', StringType(), True),
        StructField('missed_bytes', IntegerType(), True),
        StructField('history', StringType(), True),
        StructField('orig_pkts', IntegerType(), True),
        StructField('orig_ip_bytes', IntegerType(), True),
        StructField('resp_pkts', IntegerType(), True),
        StructField('resp_ip_bytes', IntegerType(), True),
        StructField('tunnel_parents', StringType(), True)
    ])

ecar_bro_schema = StructType([
        StructField('action', StringType(), True),
        StructField('actorID', StringType(), True),
        StructField('hostname', StringType(), True),
        StructField('id', StringType(), True),
        StructField('object', StringType(), True),
        StructField('objectID', StringType(), True),
        StructField('pid', IntegerType(), True),
        StructField('ppid', IntegerType(), True),
        StructField('principal', StringType(), True),
        StructField('properties', StructType([
            StructField('acuity_level', StringType(), True),
            StructField('bro_uid', StringType(), True),
            StructField('dest_ip', StringType(), True),
            StructField('dest_port', IntegerType(), True),
            StructField('direction', StringType(), True),
            StructField('image_path', StringType(), True),
            StructField('l4protocol', StringType(), True),
            StructField('src_ip', StringType(), True),
            StructField('src_port', IntegerType(), True),
            ])),
        StructField('tid', IntegerType(), True),
        StructField('timestamp', TimestampType(), True)
    ])

ecar_schema = StructType([
        StructField('action', StringType(), True),
        StructField('actorID', StringType(), True),
        StructField('hostname', StringType(), True),
        StructField('id', StringType(), True),
        StructField('object', StringType(), True),
        StructField('objectID', StringType(), True),
        StructField('pid', IntegerType(), True),
        StructField('ppid', IntegerType(), True),
        StructField('principal', StringType(), True),
        StructField('properties', StructType([
            StructField('acuity_level', StringType(), True),
            StructField('base_address', StringType(), True),
            StructField('command_line', StringType(), True),
            StructField('context_info', StringType(), True),
            StructField('data', StringType(), True),
            StructField('dest_port', IntegerType(), True),
            StructField('direction', StringType(), True),
            StructField('end_time', TimestampType(), True),
            StructField('file_path', StringType(), True),
            StructField('image_path', StringType(), True),
            StructField('info_class', StringType(), True),
            StructField('key', StringType(), True),
            StructField('l4protocol', StringType(), True),
            StructField('logon_id', StringType(), True),
            StructField('module_path', StringType(), True),
            StructField('new_path', StringType(), True),
            StructField('parent_image_path', StringType(), True),
            StructField('path', StringType(), True),
            StructField('payload', StringType(), True),
            StructField('privileges', StringType(), True),
            StructField('requesting_domain', StringType(), True),
            StructField('requesting_logon_id', StringType(), True),
            StructField('requesting_user', StringType(), True),
            StructField('sid', StringType(), True),
            StructField('size', StringType(), True),
            StructField('src_ip', StringType(), True),
            StructField('src_pid', IntegerType(), True),
            StructField('src_port', IntegerType(), True),
            StructField('src_tid', StringType(), True),
            StructField('stack_base', StringType(), True),
            StructField('stack_limit', StringType(), True),
            StructField('start_address', StringType(), True),
            StructField('start_time', TimestampType(), True),
            StructField('subprocess_tag', StringType(), True),
            StructField('task_name', StringType(), True),
            StructField('task_pid', IntegerType(), True),
            StructField('task_process_uuid', StringType(), True),
            StructField('tgt_pid', IntegerType(), True),
            StructField('tgt_pid_uuid', StringType(), True),
            StructField('tgt_tid', IntegerType(), True),
            StructField('type', StringType(), True),
            StructField('user', StringType(), True),
            StructField('user_name', StringType(), True),
            StructField('user_stack_base', StringType(), True),
            StructField('user_stack_limit', StringType(), True),
            StructField('value', StringType(), True)
            ])),
        StructField('tid', IntegerType(), True),
        StructField('timestamp', TimestampType(), True)
    ])

In [8]:
def get_ecar_files():
    env = 'prod'
    type = 'car'
    paths=['23Sep19-red',
           '24Sep19',
           '25Sep'
          ]
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(
        Bucket = 'sapient-bucket-raw',
        Prefix = f'{env}/')

    all_files = []
    for p in paths:
        files = []
        for content in response.get('Contents', []):
            files.append(content['Key'])
        files = [f"{s3_url_raw}/" + f for f in files if p in f]
        all_files = all_files + files
    return all_files

In [9]:
def loadAndCheckpoint(type='ecar-bro', env='prod', size='small'):
    """
    type: ecar, ecar-bro, bro, labels
    This function reads a file from json or log text and writes it as a parquet.
    """
    ecar_fil = [("FLOW"), ("PROCESS"), ("FILE"),("SHELL")]
    if size == 'small':
        # 1 million
        read_lim = 1000000
    elif size == 'medium':
        # 100 million
        read_lim = 100000000
    elif size == 'large':
        # 1 billion
        read_lim = 1000000000  
    elif size == 'all':
        # 1 billion
        read_lim = 'all' 
    start_time = time.time()
    if type in ('ecar', 'car'):
        if env == 'prod':
            ecar_files = get_ecar_files()
                #TODO: new filter for time
            df = spark.read.schema(ecar_schema).json(ecar_files).filter(col("object").isin(ecar_fil) & 
                                                                       (dayofmonth(col('timestamp')) == 23)).cache()
        else:
            df = spark.read.schema(ecar_schema).json(f"{s3_url_raw}/{env}/{type}/**/**/**/*.json").filter(col("object").isin(ecar_fil)).cache()
        if size == 'all':
            pass
        else: 
            df = df.limit(read_lim)
        print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " -- read time: --- %s seconds ---" % (time.time() - start_time))
        #print(time.strftime('%l:%M%p %Z on %b %d, %Y') + f" -- Your new dataframe has {df.count():,} rows.")
        start_time = time.time()
        df = df.select(*df.columns, "properties.*").drop('properties')
        df = df.withColumn('event_minute', minute(col('timestamp'))) \
               .withColumn('event_day', dayofmonth(col('timestamp'))) \
               .withColumn('event_hour', hour(col('timestamp')))
        print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " -- schema expansion time: --- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()
        # numPartitions = 16000
        # df = df.repartition(numPartitions)
        df.write.option("maxRecordsPerFile", 300000).mode("overwrite").partitionBy("event_day", "event_hour", "event_minute").parquet(f"{s3_url_refined}/{env}/ecar/{size}")
    elif type in ('ecar-bro','car-bro'):
        df = spark.read.schema(ecar_bro_schema).json(f"{s3_url_raw}/{env}/{type}/**/**/**/*.json").cache()
        # this will extract and flatten nested properties column
        if size == 'all':
            pass
        else: 
            df = df.limit(read_lim)
        print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " -- read time: --- %s seconds ---" % (time.time() - start_time))
        #print(time.strftime('%l:%M%p %Z on %b %d, %Y') + f" -- Your new dataframe has {df.count():,} rows.")
        start_time = time.time()
        df = df.select(*df.columns, "properties.*").drop('properties')
        df = df.withColumn('event_minute', minute(col('timestamp'))) \
               .withColumn('event_day', dayofmonth(col('timestamp'))) \
               .withColumn('event_hour', hour(col('timestamp')))
        df.write.option("maxRecordsPerFile", 300000).mode("overwrite").partitionBy("event_day", "event_hour").parquet(f"{s3_url_refined}/{env}/ecar-bro/{size}")
    elif type == 'bro':
        df = spark.read.schema(bro_schema).csv(f"{s3_url_raw}/{env}/**/**/conn*.log", sep="\t", comment="#", header=False).cache()
        if size == 'all':
            pass
        else: 
            df = df.limit(read_lim)
        df = df.toDF(*bro_cols_conn)
        print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " -- read time: --- %s seconds ---" % (time.time() - start_time))
        #print(time.strftime('%l:%M%p %Z on %b %d, %Y') + f" -- Your new dataframe has {df.count():,} rows.")
        start_time = time.time()
        df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/bro/{size}")
    elif type == 'labels':
        df = spark.read.csv(f"{s3_url_raw}/{type}/*.csv", sep=",", header=True).cache()
        df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/labels")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " -- write time: --- %s seconds ---" % (time.time() - start_time))
    df.unpersist()

In [10]:
env='prod'
size='small'
# this is here because the prod ecar/ecar-bro are missing the "e"s and is a short term fix
if env == 'prod':
    ftype = 'car'
else:
    ftype = 'ecar'

In [31]:
loadAndCheckpoint('labels', env=env)

write time: --- 9.058502435684204 seconds ---


In [11]:
loadAndCheckpoint(f'{ftype}-bro', env=env, size=size)

read time: --- 9.37377119064331 seconds ---


write time: --- 153.34089350700378 seconds ---


In [11]:
loadAndCheckpoint(f'{ftype}', env=env, size=size)

12:44AM UTC on Mar 04, 2023 -- read time: --- 14.87155532836914 seconds ---
12:44AM UTC on Mar 04, 2023 -- schema expansion time: --- 0.4012305736541748 seconds ---


ERROR:root:Exception while sending command.                    (19 + 4) / 13021]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/s

Py4JError: An error occurred while calling o101.parquet

In [13]:
size='medium'

In [14]:
loadAndCheckpoint(f'{ftype}-bro', env=env, size=size)

read time: --- 3.033254861831665 seconds ---


write time: --- 725.4632656574249 seconds ---


In [15]:
loadAndCheckpoint(f'{ftype}', env=env, size=size)

read time: --- 0.5497806072235107 seconds ---
schema expansion time: --- 0.06387162208557129 seconds ---


write time: --- 2129.4056890010834 seconds ---


In [16]:
size='large'

In [17]:
loadAndCheckpoint(f'{ftype}-bro', env=env, size=size)

read time: --- 3.0137670040130615 seconds ---


write time: --- 752.6250596046448 seconds ---


In [18]:
loadAndCheckpoint(f'{ftype}', env=env, size=size)

read time: --- 0.584460973739624 seconds ---
schema expansion time: --- 0.06183433532714844 seconds ---


write time: --- 10365.53863453865 seconds ---


In [11]:
size='all'

In [33]:
loadAndCheckpoint(f'{ftype}-bro', env=env, size=size)

read time: --- 2.7087810039520264 seconds ---


write time: --- 2072.032541513443 seconds ---


In [ ]:
loadAndCheckpoint(f'{ftype}', env=env, size=size)

 5:32AM UTC on Feb 28, 2023 -- read time: --- 7.466861724853516 seconds ---


 5:32AM UTC on Feb 28, 2023 -- Your new dataframe has 2,898,070,614 rows.
 6:13AM UTC on Feb 28, 2023 -- schema expansion time: --- 0.15707945823669434 seconds ---
